In [ ]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)


req = ogd_api.Request(
    collection="ogd-forecasting-icon-ch2",
    variable="T",
    reference_datetime=today_midnight_utc,
    perturbed=False,
    horizon=f"P0DT0H",
)

In [ ]:
from earthkit.data import config
config.set("cache-policy", "temporary")

T = ogd_api.get_from_ogd(req)
T

In [ ]:
from meteodatalab import ogd_api
from meteodatalab import grib_decoder, data_source
from meteodatalab.operators.destagger import destagger
from meteodatalab.operators.vertical_interpolation import interpolate_k2any, TargetCoordinates, TargetCoordinatesAttrs

# access vertical constant
url = ogd_api.get_collection_asset_url(
    collection_id="ch.meteoschweiz.ogd-forecasting-icon-ch2",
    asset_id="vertical_constants_icon-ch2-eps.grib2"
)
ds = grib_decoder.load(
    source=data_source.URLDataSource(urls=[url]),
    request={"param": "HHL"},
    geo_coords=lambda uuid: {}
)
HFL = destagger(ds["HHL"].squeeze(drop=True), "z")

# interpolate to 1000 amsl
attrs = TargetCoordinatesAttrs("height_above_mean_sea_level", "height above the mean sea level", "m", "up")
target_coords = TargetCoordinates("heightAboveSea", [1000], attrs)

T_int = interpolate_k2any(
    field=T,
    mode="high_fold",
    tc_field=HFL,
    tc=target_coords,
    h_field=HFL
    )

In [ ]:
coords = T_int.coords["z"]
attrs = T_int.attrs["vcoord_type"]
print(f"\033[1mz \033[0m(vertical model levels): {coords.values}")
print(f"\033[1mvcoord_type \033[0m(type of vertical coordinate): {attrs}")
T_int